In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# Cargar el conjunto de datos
data = pd.read_csv('/content/drive/MyDrive/USFX/SIS420-IA/datasets/emnist-letters-train.csv')

In [26]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88799 entries, 0 to 88798
Columns: 785 entries, 23 to 0.415
dtypes: int64(785)
memory usage: 531.8 MB


In [28]:
# Dividir los datos en características (X) y etiquetas (y)
X = data.drop('23', axis=1).values
y = data['23'].values

# Codificar las etiquetas en valores numéricos
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# Definir una clase personalizada para el conjunto de datos
#Esta clase se utiliza para crear instancias de conjuntos de datos personalizados
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Crear instancias del conjunto de datos
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [30]:
# Definir el modelo del perceptrón
class Perceptron(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size, num_classes) #crea una capa lineal

    def forward(self, x):
        x = self.fc(x)
        return x

# Definir los parámetros del modelo
input_size = X.shape[1]
num_classes = len(label_encoder.classes_)

# Crear una instancia del modelo
model = Perceptron(input_size, num_classes)

In [31]:
# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss() #pérdida de entropía cruzada
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Definir los dataloaders para el entrenamiento y la prueba
batch_size = 20 #número de muestras que se utilizarán en cada iteración durante el entrenamiento
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [32]:
# Entrenamiento del modelo
#implementa un bucle de entrenamiento que itera sobre las épocas y los lotes de datos cargados
#se realiza la propagación hacia adelante, se calcula la pérdida, se realiza la propagación
#hacia atrás y se actualizan los parámetros del modelo utilizando un optimizador
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 2610.7617
Epoch [2/10], Loss: 2300.9250
Epoch [3/10], Loss: 1379.2004
Epoch [4/10], Loss: 2612.3533
Epoch [5/10], Loss: 1667.2133
Epoch [6/10], Loss: 1815.5510
Epoch [7/10], Loss: 2139.9126
Epoch [8/10], Loss: 879.6154
Epoch [9/10], Loss: 555.6930
Epoch [10/10], Loss: 2177.9895


In [33]:
# Evaluación del modelo
#calcula la precisión dividiendo el número total de etiquetas predichas
#correctamente entre el número total de etiquetas en el conjunto de prueba
model.eval() #coloca el modelo en modo de evaluación
with torch.no_grad(): #Esto desactiva el seguimiento de gradientes durante la evaluación
    correct = 0
    total = 0
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}') #muestra la precisión del modelo

Accuracy: 0.5618
